In [29]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from glob import glob
import numpy as np

In [30]:
csv_files = glob(os.path.join('./data/benign_or_attack', "*.csv"))
print(csv_files)

['./data/benign_or_attack\\BENIGN_Bot.csv', './data/benign_or_attack\\BENIGN_PortScan.csv', './data/benign_or_attack\\DDoS_BENIGN.csv', './data/benign_or_attack\\DoS_BENIGN.csv', './data/benign_or_attack\\WebAttacks.csv']


In [31]:
data_frames = []

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        print(f"Файл {csv_file}: {df.shape[0]} строк, {df.shape[1]} столбцов")
        data_frames.append(df)
    except Exception as e:
        print(f"Ошибка при чтении {csv_file}: {e}")

if not data_frames:
    raise ValueError("Не удалось прочитать ни один CSV-файл.")

Файл ./data/benign_or_attack\BENIGN_Bot.csv: 191033 строк, 79 столбцов
Файл ./data/benign_or_attack\BENIGN_PortScan.csv: 286467 строк, 79 столбцов
Файл ./data/benign_or_attack\DDoS_BENIGN.csv: 225745 строк, 79 столбцов
Файл ./data/benign_or_attack\DoS_BENIGN.csv: 692703 строк, 79 столбцов
Файл ./data/benign_or_attack\WebAttacks.csv: 170366 строк, 79 столбцов


In [38]:
data = pd.concat(data_frames, ignore_index=True)
data = data.dropna()
data = data.fillna(0).replace([np.inf, -np.inf], 0)
data.columns = data.columns.str.strip()
print(f"Объединённый DataFrame из {len(csv_files)} файла: {data.shape[0]} строк, {data.shape[1]} столбцов")

Объединённый DataFrame из 5 файла: 1565239 строк, 79 столбцов


In [39]:
print("\nРаспределение классов до преобразования меток:")
print(data['Label'].value_counts(normalize=True) * 100)

data['Label'] = data['Label'].apply(
    lambda x: 'benign' if 'BENIGN' in str(x).upper() else 'attack'
)


Распределение классов до преобразования меток:
Label
BENIGN                        65.319929
DoS Hulk                      14.702164
PortScan                      10.153721
DDoS                           8.179390
DoS GoldenEye                  0.657599
DoS slowloris                  0.370295
DoS Slowhttptest               0.351320
Bot                            0.125604
Web Attack � Brute Force       0.096279
Web Attack � XSS               0.041655
Web Attack � Sql Injection     0.001342
Heartbleed                     0.000703
Name: proportion, dtype: float64


In [40]:
_, reduced_data = train_test_split(
    data,
    test_size=0.1,
    stratify=data['Label']
)

In [43]:
print(reduced_data.columns)
reduced_data = reduced_data.drop('Fwd Header Length.1', axis=1)

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

In [44]:
print(f"\nУменьшённый датасет: {reduced_data.shape[0]} строк, {reduced_data.shape[1]} столбцов")

# Проверяем распределение классов после уменьшения
print("\nРаспределение классов после уменьшения:")
reduced_class_counts = data['Label'].value_counts(normalize=True) * 100
print(reduced_class_counts)


Уменьшённый датасет: 156524 строк, 78 столбцов

Распределение классов после уменьшения:
Label
benign    65.319929
attack    34.680071
Name: proportion, dtype: float64


In [45]:
# Сохраняем уменьшённый датасет
output_file = os.path.join('./data/benign_or_attack_merged/', "benign_or_attack.csv")
reduced_data.to_csv(output_file, index=False)
print(f"\nУменьшённый датасет сохранён в {output_file}")


Уменьшённый датасет сохранён в ./data/benign_or_attack_merged/benign_or_attack.csv
